- https://huggingface.co/microsoft/Phi-4-mini-instruct

In [ ]:
import json
import os
from textwrap import dedent

import gait as G
import openai
from faker import Faker
from pydantic import BaseModel, Field
from rich.pretty import pprint
from vllm import LLM, SamplingParams

In [ ]:
llm = LLM(
    # model="microsoft/Phi-4-mini-instruct",
    # model="microsoft/phi-4",
    # model="meta-llama/Llama-3.2-3B-Instruct",
    model="Qwen/Qwen2.5-3B-Instruct",
    trust_remote_code=True,
    max_model_len=16384,
    dtype="float16",  # MacOS
)

In [ ]:
sampling_params = SamplingParams(
    max_tokens=2048,
    temperature=0.0,
)

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful AI assistant.",
    },
    {
        "role": "user",
        "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
    },
    {
        "role": "assistant",
        "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey.",
    },
    {
        "role": "user",
        "content": "What about solving an 2x + 3 = 7 equation?",
    },
]


output = llm.chat(
    messages=messages,
    sampling_params=sampling_params,
)
print(output[0].outputs[0].text)

In [ ]:
system = dedent(
    """
You are an expert at generating simple JSON documents that recommend one of three possible solutions.

A possible >>>SOLUTION<<< is either:

1. ROUTE
2. GEOCODE
3. TEMPERATURE

ROUTE is when you have to route between two addresses, places or cities on a road network.
GEOCODE is when you have to convert an address, city, state, zip code or country to an x,y point.
TEMPERATURE is when you have to tell the temperature or tell the weather for any place on earth.

Make sure to:

1. Not generate ANY code in the answer.
2. Only output a simple JSON document like {"oper":>>>SOLUTION<<<}
3. Only recommend ONE of the possible answers.
4. Infer and learn the correct solution based on the input.
"""
).strip()

In [ ]:
messages = [
    G.s_message(system),
    G.u_message("Show the route to San Diegeo from San Francisco"),
    # G.u_message("What is the location of Florence, Italy?"),
    # G.u_message("What is the current temperature of Florence, Italy?"),
]
output = llm.chat(
    messages=messages,
    sampling_params=sampling_params,
    use_tqdm=False,
)
# pprint(output[0].outputs[0], expand_all=True)

In [ ]:
text = output[0].outputs[0].text.replace("\n", "").replace("```json","").replace("```","")
print(text)

In [ ]:
pprint(
    json.loads(text),
    expand_all=True,
)

In [ ]:
fake = Faker()

In [ ]:
class GetLatLon(BaseModel):
    """Get the latitude and longitude of a given location."""

    location: str = Field(
        ...,
        description="A location, can be a place, city, state, zipcode, state or country.",
    )

    def __call__(self, *args, **kwargs):
        lon = fake.longitude()
        lat = fake.latitude()
        return {
            "location": self.location,
            "longitude": float(lon),
            "latitude": float(lat),
        }

In [ ]:
class GetRoute(BaseModel):
    """Get the route between a starting latitude/longitude location and an ending latitude/longitude location."""

    lon1: float = Field(
        ...,
        description="The route starting longitude.",
    )
    lat1: float = Field(
        ...,
        description="The route starting latitude.",
    )
    lon2: float = Field(
        ...,
        description="The route ending longitude.",
    )
    lat2: float = Field(
        ...,
        description="The route ending latitude.",
    )

    def __call__(self, *args, **kwargs):
        lon = fake.longitude()
        lat = fake.latitude()

        if "scratchpad" in kwargs:
            kwargs["scratchpad"]["GetRoute"] = {"lon": lon, "lat": lat}

        return {
            "route": f"{self.lat1},{self.lon1} ---> {self.lat2},{self.lon2}",
        }

In [ ]:
class GetCurrentTemperature(BaseModel):
    """Get the current temperature at a given location."""

    location: str = Field(
        ...,
        description="A location, can be a place, city, state, zipcode, state or country.",
    )
    celsius_or_fahrenheit: str = Field(
        ...,
        description="The temperature in either 'C' for Celsius, or 'F' for Fahrenheit.",
    )

    def __call__(self, *args, **kwargs):
        temp = random.uniform(-5, 40)
        return {
            self.location: f"{temp:.1f}{self.celsius_or_fahrenheit}",
        }

In [ ]:
tools = [
    openai.pydantic_function_tool(_)
    for _ in [
        GetCurrentTemperature,
        GetLatLon,
        GetRoute,
    ]
]

# pprint(tools, expand_all=True)

In [ ]:
system = """
You are an AI expert in geo-spatial data analysis with access to geo-spatial tools.
You are tasked to answer a user >>>question<<<.
You will run in a loop.
At the end of the loop you output an answer.

Here are the rules you should always follow to solve your task:
- ALWAYS use the right arguments for the tools. NEVER use variable names.
- ALWAYS use the values instead.
- NEVER re-do a tool call that you previously did with the exact same arguments.

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000.
""".strip()

# prompt = f"""
# What's the current temperature in {fake.city()} in celsius and {fake.city()} in fahrenheit?
# And what is the route between them?
# """

# prompt = dedent(
#     f"""
# >>>What's the current temperature in {fake.city()} in celsius? and what are its coordinates?<<
# """
# ).strip()

prompt = dedent(
    f"""
>>>What's the route between {fake.city()} and {fake.city()}?<<
"""
).strip()


messages = [
    G.s_message(system),
    G.u_message(prompt),
]

In [ ]:
outputs = llm.chat(
    messages,
    sampling_params=sampling_params,
    tools=tools,
    use_tqdm=False,
)
output = outputs[0].outputs[0].text.strip()

# append the assistant message
messages.append(
    {
        "role": "assistant",
        "content": output,
    }
)

In [ ]:
print(output)

In [ ]:
pprint(json.loads(output), expand_all=True)